In [1]:
import numpy as np
import pandas as pd
from typing import Set
import networkx as nx
from typing import List, Tuple
import string

Par_Paired

In [2]:
def find_nseg(file_path: str, target_id: int) -> List[int]:
    '''Find upstream segments in a river network given
    in the form of a text file between "tosegment" and "#### 
    these represent the nseg index values to replace with parameters, the target ID is included in the list
    because the gauge is assumed to be at the end of the segments
    
    Parameters
    ----------
    file_path: str
        Path to the text file containing river segment information
    target_id: int
        Target ID for which upstream segments are desired
    
    Returns
    -------
    nodes: list of int
        IDs of upstream or ancestor segments
    '''
    with open(file_path, 'r') as file:
        lines = file.readlines()

    # Convert lines to a NumPy array
    data = np.array([line.strip() for line in lines])

    # Find the index of 'tosegment'
    start_index = np.where(data == 'tosegment')[0]

    if len(start_index) == 0:
        print("Target string 'tosegment' not found in the file.")
        return []

    start_index = start_index[0]

    # Find the index of the next '####'
    end_index = np.where(data == '####')[0]

    if len(end_index) == 0:
        print("No '####' found after 'tosegment'.")
        return []

    end_index = end_index[np.where(end_index > start_index)]

    if len(end_index) == 0:
        print("No '####' found after 'tosegment'.")
        return []

    end_index = end_index[0]

    start_index += 5
    
    result_array = data[start_index:end_index]

    # Create a DataFrame
    df = pd.DataFrame({'upstream_segments': result_array}, index=np.arange(1, len(result_array) + 1))
    
    # Convert index to strings
    df.index = df.index.astype(str)

    # Creating a DiGraph out of `df` object
    riv_graph = nx.from_pandas_edgelist(df.reset_index(), source='index', target='upstream_segments', create_using=nx.DiGraph)
    
    # Return nodes as a list of integers, including the target_id
    nodes = nx.ancestors(riv_graph, str(target_id))
    nodes_list = list(map(int, nodes)) + [target_id]

    return nodes_list

In [3]:
def remove_duplicates(list1, list2):
    '''Find unique values from two lists.
    
    Parameters
    ----------
    list1: list of int
        First list of integers
    list2: list of int
        Second list of integers
    
    Returns
    -------
    unique_values: list of int
        List of unique values from both lists
    '''
    set1 = set(list1)
    set2 = set(list2)
    
    # Find unique values from both sets
    unique_values = list(set1.symmetric_difference(set2))
    
    return unique_values

In [4]:
def combine_nseg(list1, list2):
    '''Merge two lists and find unique values.
    
    Parameters
    ----------
    list1: list of int
        First list of integers
    list2: list of int
        Second list of integers
    
    Returns
    -------
    unique_values: list of int
        List of unique values from both lists
    '''
    merged_list = list1 + list2
    unique_values = list(set(merged_list))
    
    return unique_values


In [5]:
def find_nhru(file_path: str, nseg: list) -> List[int]:
    ''' find the corresponding nhru index positions to the nseg
    
    Parameters
    ----------
    file_path: str
        Path to the text file containing river segment information
        
    nseg: list of index positions of upstream river segments
    
    Returns
    -------
    nhru: list of hru values corresponding to the segments
    
    '''
    with open(file_path, 'r') as file:
        lines = file.readlines()

    # Convert lines to a NumPy array
    data = np.array([line.strip() for line in lines])

    # Find the index of 'tosegment'
    start_index = np.where(data == 'hru_segment')[0]

    if len(start_index) == 0:
        print("Target string 'tosegment' not found in the file.")
        return []

    start_index = start_index[0]

    # Find the index of the next '####'
    end_index = np.where(data == '####')[0]

    if len(end_index) == 0:
        print("No '####' found after 'tosegment'.")
        return []

    end_index = end_index[np.where(end_index > start_index)]

    if len(end_index) == 0:
        print("No '####' found after 'tosegment'.")
        return []

    end_index = end_index[0]

    start_index += 5
    
    result_array = data[start_index:end_index]
    
    # Create a DataFrame
    df = pd.DataFrame({'corresponding hru': result_array}, index=np.arange(1, len(result_array) + 1))

    # Find corresponding nhru values for each nseg
    nhru_values = []
    for seg in nseg:
        # Find index positions where seg is present in the first column
        indices = np.where(df['corresponding hru'] == str(seg))[0] + 1  # Adding 1 to convert to 1-based index
        nhru_values.extend(indices)

    return nhru_values


In [6]:
def write_nseg(array: np.ndarray, dataframe: pd.DataFrame, nhru_results: List[int], updated_strings: List[List[str]]) -> np.ndarray:
    '''Filter the DataFrame based on the 'Count' column values of 908 or 909.
    
    Parameters
    ----------
    array: np.ndarray
        NumPy array (to be updated).
    dataframe: pd.DataFrame
        DataFrame to be filtered.
    nhru_results: List[int]
        List of nhru values.
    updated_strings: List[List[str]]
        2D List to store updated strings (each row: [original_string, numerical_value, updated_string]).
    
    Returns
    -------
    updated_array: np.ndarray
        Updated NumPy array with index numbers rewritten as strings.
    '''
    # Filter the DataFrame based on 'Count' column values
    filtered_dataframe = dataframe[dataframe['Count'].isin([448, 449])]
    
    # Iterate through strings in the index of the filtered DataFrame
    for string_to_find in filtered_dataframe.index:
        # Find the index of 'tosegment'
        start_index = np.where(array == string_to_find)[0]

        if len(start_index) == 0:
            print(f"Target string '{string_to_find}' not found in the array.")
            continue

        start_index = start_index[0]

        # Find the index of the next '####'
        end_index = np.where(array == '####')[0]

        if len(end_index) == 0:
            print("No '####' found after the target string.")
            continue

        end_index = end_index[np.where(end_index > start_index)]

        if len(end_index) == 0:
            print("No '####' found after the target string.")
            continue

        end_index = end_index[0]

        # Skip the first 4 lines for nseg
        start_index += 5

        # Save the range between start and end index to a NumPy array
        result_array = np.arange(start_index, end_index + 1)

        # Iterate through the list of nhru_results and update the corresponding indices
        for nhru_value in nhru_results:
            # Map nhru_value to the corresponding index in result_array
            index_to_update = nhru_value - 1

            if index_to_update < 0 or index_to_update >= len(result_array):
                print(f"Index value '{nhru_value}' is out of range for '{string_to_find}'. Skipping.")
                continue

            # Save the numerical value being overwritten
            original_value = array[result_array[index_to_update]]

            # Update the array with the new string format
            updated_value = f'___{string_to_find}{nhru_value}_A'
            array[result_array[index_to_update]] = updated_value

            # Append the updated string to the 2D list
            updated_strings.append([string_to_find, original_value, updated_value])

    return array

In [7]:
def write_nhru(array: np.ndarray, dataframe: pd.DataFrame, nhru_results: List[int], updated_strings: List[List[str]]) -> np.ndarray:
    '''Filter the DataFrame based on the 'Count' column values of 908 or 909.
    
    Parameters
    ----------
    array: np.ndarray
        NumPy array (to be updated).
    dataframe: pd.DataFrame
        DataFrame to be filtered.
    nhru_results: List[int]
        List of nhru values.
    updated_strings: List[List[str]]
        2D List to store updated strings (each row: [original_string, numerical_value, updated_string]).
    
    Returns
    -------
    updated_array: np.ndarray
        Updated NumPy array with index numbers rewritten as strings.
    '''
    # Filter the DataFrame based on 'Count' column values
    filtered_dataframe = dataframe[dataframe['Count'].isin([908, 909])]
    
    # Iterate through strings in the index of the filtered DataFrame
    for string_to_find in filtered_dataframe.index:
        # Find the index of 'tosegment'
        start_index = np.where(array == string_to_find)[0]

        if len(start_index) == 0:
            print(f"Target string '{string_to_find}' not found in the array.")
            continue

        start_index = start_index[0]

        # Find the index of the next '####'
        end_index = np.where(array == '####')[0]

        if len(end_index) == 0:
            print("No '####' found after the target string.")
            continue

        end_index = end_index[np.where(end_index > start_index)]

        if len(end_index) == 0:
            print("No '####' found after the target string.")
            continue

        end_index = end_index[0]

        # Skip the first 4 lines for nhru
        start_index += 5

        # Save the range between start and end index to a NumPy array
        result_array = np.arange(start_index, end_index + 1)

        # Iterate through the list of nhru_results and update the corresponding indices
        for nhru_value in nhru_results:
            # Map nhru_value to the corresponding index in result_array
            index_to_update = nhru_value - 1

            if index_to_update < 0 or index_to_update >= len(result_array):
                print(f"Index value '{nhru_value}' is out of range for '{string_to_find}'. Skipping.")
                continue

            # Save the numerical value being overwritten
            original_value = array[result_array[index_to_update]]

            # Update the array with the new string format
            updated_value = f'___{string_to_find}{nhru_value}_A'
            array[result_array[index_to_update]] = updated_value

            # Append the updated string to the 2D list
            updated_strings.append([string_to_find, original_value, updated_value])

    return array

In [8]:
def write_nmonths(array: np.ndarray, dataframe: pd.DataFrame, nhru_results: List[int], updated_strings: List[List[str]]) -> np.ndarray:
    '''Filter the DataFrame based on the 'Count' column values of 908 or 909.
    
    Parameters
    ----------
    array: np.ndarray
        NumPy array (to be updated).
    dataframe: pd.DataFrame
        DataFrame to be filtered.
    nhru_results: List[int]
        List of nhru values.
    updated_strings: List[List[str]]
        2D List to store updated strings (each row: [original_string, numerical_value, updated_string]).
    
    Returns
    -------
    updated_array: np.ndarray
        Updated NumPy array with index numbers rewritten as strings.
    '''
    # Filter the DataFrame based on 'Count' column values
    filtered_dataframe = dataframe[dataframe['Count'].isin([10908, 10909])]
    
    # Iterate through strings in the index of the filtered DataFrame
    for string_to_find in filtered_dataframe.index:
        # Find the index of 'tosegment'
        start_index = np.where(array == string_to_find)[0]

        if len(start_index) == 0:
            print(f"Target string '{string_to_find}' not found in the array.")
            continue

        start_index = start_index[0]

        # Find the index of the next '####'
        end_index = np.where(array == '####')[0]

        if len(end_index) == 0:
            print("No '####' found after the target string.")
            continue

        end_index = end_index[np.where(end_index > start_index)]

        if len(end_index) == 0:
            print("No '####' found after the target string.")
            continue

        end_index = end_index[0]

        # Skip the first 5 lines for nmonths
        start_index += 6

        # Save the range between start and end index to a NumPy array
        result_array = np.arange(start_index, end_index + 1)

        # Iterate through the list of nhru_results and update the corresponding indices
        for nhru_value in nhru_results:
            # Map nhru_value to the corresponding index in result_array
            index_to_update = (nhru_value - 1) * 12

            if index_to_update < 0 or index_to_update + 11 >= len(array):
                print(f"Index value '{nhru_value}' is out of range for '{string_to_find}'. Skipping.")
                continue

            # Save the numerical values being overwritten
            original_values = array[result_array[index_to_update:index_to_update + 12]]

            for i in range(12):
                # Use letters A-L at the end instead of double digits
                updated_value = f'___{string_to_find}{nhru_value}_{string.ascii_uppercase[i]}'
                array[result_array[index_to_update + i]] = updated_value

                # Append the updated string to the 2D list
                updated_strings.append([string_to_find, original_values[i], updated_value])

    return array

In [9]:
def write_alt_input(array: np.ndarray, dataframe: pd.DataFrame, nhru_results: List[int], updated_strings: List[List[str]]) -> pd.DataFrame:
    '''
    
    Parameters
    ----------
    array: np.ndarray
        NumPy array (to be updated).
    dataframe: pd.DataFrame
        DataFrame to be filtered.
    nhru_results: List[int]
        List of nhru values.
    updated_strings: List[List[str]]
        2D List to store updated strings (each row: [original_string, numerical_value, updated_string]).
    
    Returns
    -------
    new_dataframe: pd.DataFrame
        New DataFrame with 'nhru' as the index and columns named after the row index in the filtered DataFrame, filled with 1.
    '''
    # Filter the DataFrame based on 'Count' column values
    filtered_dataframe = dataframe[dataframe['Count'].isin([10908, 10909])]
    
    # Create a new DataFrame with 'nhru' as the index and specified columns
    new_dataframe = pd.DataFrame(index=nhru_results, columns=filtered_dataframe.index.astype(str))
    
    # Fill all values in the new DataFrame with 1
    new_param= new_dataframe.fillna(1)
    
        # Extract column names as a list
    column_names = new_dataframe.columns.tolist()
    
    # Iterate through each column and populate with strings
    for column_name in new_dataframe.columns:
        for row_index in new_dataframe.index:
            value = f'___adjust_{column_name}_{row_index}_A'
            new_dataframe.at[row_index, column_name] = value
            updated_strings.append([column_name, new_dataframe.at[row_index, column_name], value])
    
    return new_param, new_dataframe

In [10]:
def obsin_segment(array: np.ndarray, seg: int, obs: int) -> np.ndarray:
    '''Replace the inflow to a segment with observed flow.
    
    Parameters
    ----------
    array: np.ndarray
        NumPy array (to be updated).
    seg: int
        Segment to replace.
    obs: int
        Index of observed flow.
    
    Returns
    -------
    updated_array: np.ndarray
        Updated NumPy array with index numbers replaced by observed flow.
    '''

    # Find the index of 'obsin_segment'
    start_index = np.where(array == 'obsin_segment')[0]

    if len(start_index) == 0:
        print("Target string 'obsin_segment' not found in the array.")
        return array

    start_index = start_index[0]

    # Find the index of the next '####'
    end_index = np.where(array == '####')[0]

    if len(end_index) == 0:
        print("No '####' found after 'obsin_segment'.")
        return array

    end_index = end_index[np.where(end_index > start_index)]

    if len(end_index) == 0:
        print("No '####' found after 'obsin_segment'.")
        return array

    end_index = end_index[0]

    # Skip the first 4 lines for nseg
    start_index += 5

    # Save the range between start and end index to a NumPy array
    result_array = np.arange(start_index, end_index + 1)

    # Map seg to the corresponding index in result_array
    index_to_update = seg - 1

    if index_to_update < 0 or index_to_update >= len(result_array):
        print(f"Index value '{seg}' is out of range for 'obsin_segment'. Skipping.")
        return array

    # Update the array with the observed flow value
    array[result_array[index_to_update]] = obs

    return array


In [11]:
def write_ostin_table(ostin_table: pd.DataFrame, updated_strings: np.ndarray) -> pd.DataFrame:
    '''
    Generate a new DataFrame based on specified conditions.

    Parameters
    ----------
    ostin_table : pd.DataFrame
        DataFrame containing 'ostin_table' data.

    updated_strings : np.ndarray
        2D array containing numerical values.

    Returns
    -------
    new_dataframe : pd.DataFrame
        A new DataFrame generated based on the specified conditions.
    '''
    # Create a DataFrame from the updated_strings data
    updated_strings_df = pd.DataFrame(updated_strings, columns=['Parameter', 'Original_Value', 'Updated_String'])

    # Set the 'Parameter' column as the index for quick lookups
    ostin_table.index.name = 'Parameter'
    updated_strings_df.set_index('Parameter', inplace=True)

    # Change the original values of 0 to 0.01
    updated_strings_df['Original_Value'] = pd.to_numeric(updated_strings_df['Original_Value'], errors='coerce')
    updated_strings_df['Original_Value'].replace(0, 0.01, inplace=True)

    for parameter_name in updated_strings_df.index.unique():
        # Find corresponding values in ostin_table
        if parameter_name in ostin_table.index:
            low_bound = ostin_table.loc[parameter_name, 'low_bound']
            upper_bound = ostin_table.loc[parameter_name, 'upper_bound']

            # Check if the low_bound is -9999
            if low_bound == -9999:
                original_values = updated_strings_df.loc[parameter_name, 'Original_Value']
                updated_strings_df.loc[parameter_name, 'Low_Bound'] = 0.8 * original_values
            else:
                updated_strings_df.loc[parameter_name, 'Low_Bound'] = low_bound

            # Check if the upper_bound is -9999
            if upper_bound == -9999:
                original_values = updated_strings_df.loc[parameter_name, 'Original_Value']
                updated_strings_df.loc[parameter_name, 'Upper_Bound'] = 1.2 * original_values
            else:
                updated_strings_df.loc[parameter_name, 'Upper_Bound'] = upper_bound

    # Set the "Updated_String" column as the index
    updated_strings_df = updated_strings_df.set_index('Updated_String')

    # Add three new columns filled with 'none'
    updated_strings_df['1'] = 'none'
    updated_strings_df['2'] = 'none'
    updated_strings_df['3'] = 'none'
    
    # optionally use keyword extract
    # updated_strings_df['Original_Value']= 'extract'
    
        # Replace NaN values in 'Original_Value' with 1
    updated_strings_df['Original_Value'].fillna(1, inplace=True)
    
    return updated_strings_df

In [12]:
def update_nseg_results(array: np.ndarray, dataframe: pd.DataFrame, nhru_results: List[int], final_par: np.ndarray) -> np.ndarray:
    '''Filter the DataFrame based on the 'Count' column values of 908 or 909.
    
    Parameters
    ----------
    array: np.ndarray
        NumPy array (to be updated).
    dataframe: pd.DataFrame
        DataFrame to be filtered.
    nhru_results: List[int]
        List of nhru values.
    updated_strings: List[List[str]]
        2D List to store updated strings (each row: [original_string, numerical_value, updated_string]).
    
    Returns
    -------
    updated_array: np.ndarray
        Updated NumPy array with index numbers rewritten as strings.
    '''
    # Filter the DataFrame based on 'Count' column values
    filtered_dataframe = dataframe[dataframe['Count'].isin([448, 449])]
    
    # Iterate through strings in the index of the filtered DataFrame
    for string_to_find in filtered_dataframe.index:
        # Find the index of 'tosegment'
        start_index = np.where(array == string_to_find)[0]

        if len(start_index) == 0:
            print(f"Target string '{string_to_find}' not found in the array.")
            continue

        start_index = start_index[0]

        # Find the index of the next '####'
        end_index = np.where(array == '####')[0]

        if len(end_index) == 0:
            print("No '####' found after the target string.")
            continue

        end_index = end_index[np.where(end_index > start_index)]

        if len(end_index) == 0:
            print("No '####' found after the target string.")
            continue

        end_index = end_index[0]

        # Skip the first 4 lines for nseg
        start_index += 5

        # Save the range between start and end index to a NumPy array
        result_array = np.arange(start_index, end_index + 1)

        # Iterate through the list of nhru_results and update the corresponding indices
        for nhru_value in nhru_results:
            # Map nhru_value to the corresponding index in result_array
            index_to_update = nhru_value - 1

            if index_to_update < 0 or index_to_update >= len(result_array):
                print(f"Index value '{nhru_value}' is out of range for '{string_to_find}'. Skipping.")
                continue

            # Copy the value from the original array to the final_par array
            final_par[result_array[index_to_update]] = array[result_array[index_to_update]]

    return final_par

In [13]:
def update_nhru_results(array: np.ndarray, dataframe: pd.DataFrame, nhru_results: List[int], final_par: np.ndarray) -> np.ndarray:
    '''Filter the DataFrame based on the 'Count' column values of 908 or 909.
    
    Parameters
    ----------
    array: np.ndarray
        NumPy array (to be updated).
    dataframe: pd.DataFrame
        DataFrame to be filtered.
    nhru_results: List[int]
        List of nhru values.
    updated_strings: List[List[str]]
        2D List to store updated strings (each row: [original_string, numerical_value, updated_string]).
    
    Returns
    -------
    updated_array: np.ndarray
        Updated NumPy array with index numbers rewritten as strings.
    '''
    
    # Filter the DataFrame based on 'Count' column values
    filtered_dataframe = dataframe[dataframe['Count'].isin([908, 909])]
    
    # Iterate through strings in the index of the filtered DataFrame
    for string_to_find in filtered_dataframe.index:
        # Find the index of 'tosegment'
        start_index = np.where(final_par == string_to_find)[0]

        if len(start_index) == 0:
            print(f"Target string '{string_to_find}' not found in the array.")
            continue

        start_index = start_index[0]

        # Find the index of the next '####'
        end_index = np.where(final_par == '####')[0]

        if len(end_index) == 0:
            print("No '####' found after the target string.")
            continue

        end_index = end_index[np.where(end_index > start_index)]

        if len(end_index) == 0:
            print("No '####' found after the target string.")
            continue

        end_index = end_index[0]

        # Skip the first 4 lines for nhru
        start_index += 5

        # Save the range between start and end index to a NumPy array
        result_array = np.arange(start_index, end_index + 1)

        # Iterate through the list of nhru_results and update the corresponding indices
        for nhru_value in nhru_results:
            # Map nhru_value to the corresponding index in result_array
            index_to_update = nhru_value - 1

            if index_to_update < 0 or index_to_update >= len(result_array):
                print(f"Index value '{nhru_value}' is out of range for '{string_to_find}'. Skipping.")
                continue

            # Copy the value from the original array to the final_par array
            final_par[result_array[index_to_update]] = array[result_array[index_to_update]]

    return final_par

In [14]:
def update_nmonths_results(array: np.ndarray, dataframe: pd.DataFrame, nhru_results: List[int], final_par: np.ndarray) -> np.ndarray:
    '''Filter the DataFrame based on the 'Count' column values of 908 or 909.
    
    Parameters
    ----------
    array: np.ndarray
        NumPy array (to be updated).
    dataframe: pd.DataFrame
        DataFrame to be filtered.
    nhru_results: List[int]
        List of nhru values.
    updated_strings: List[List[str]]
        2D List to store updated strings (each row: [original_string, numerical_value, updated_string]).
    
    Returns
    -------
    updated_array: np.ndarray
        Updated NumPy array with index numbers rewritten as strings.
    '''
    # Filter the DataFrame based on 'Count' column values
    filtered_dataframe = dataframe[dataframe['Count'].isin([10908, 10909])]
    
    # Iterate through strings in the index of the filtered DataFrame
    for string_to_find in filtered_dataframe.index:
        # Find the index of 'tosegment'
        start_index = np.where(array == string_to_find)[0]

        if len(start_index) == 0:
            print(f"Target string '{string_to_find}' not found in the array.")
            continue

        start_index = start_index[0]

        # Find the index of the next '####'
        end_index = np.where(array == '####')[0]

        if len(end_index) == 0:
            print("No '####' found after the target string.")
            continue

        end_index = end_index[np.where(end_index > start_index)]

        if len(end_index) == 0:
            print("No '####' found after the target string.")
            continue

        end_index = end_index[0]

        # Skip the first 5 lines for nmonths
        start_index += 6

        # Save the range between start and end index to a NumPy array
        result_array = np.arange(start_index, end_index + 1)

        # Iterate through the list of nhru_results and update the corresponding indices
        for nhru_value in nhru_results:
            # Map nhru_value to the corresponding index in result_array
            index_to_update = (nhru_value - 1) * 12

            if index_to_update < 0 or index_to_update + 11 >= len(array):
                print(f"Index value '{nhru_value}' is out of range for '{string_to_find}'. Skipping.")
                continue


            for i in range(12):
            # Copy the value from array to final_par
                final_par[result_array[index_to_update + i]] = array[result_array[index_to_update + i]]


    return final_par

### Calibration inputs

In [15]:
# Inputs
file_path = '/home/paulc600/scratch/prms_final/par_files/template_myparam.param' # ensure its the updated par file
output_file_path = './data/paired_myparam.param'
ostin_filepath= './ostIn.txt'
target_ID= 195
upstream_ID_1= 24 # to remove hrus upstream of a previous gauge
upstream_ID_2= 429 # to remove hrus upstream of a previous gauge
updated_strings = []

### Final result inputs

In [16]:
cal_result= './mod_0/data/myparam.param' # ensure you run the script first to check KGE and update adj factor
final_par= '../par_files/template_myparam.param'
final_par_output= '/home/paulc600/scratch/prms_final/par_files/myparam.param'

### Paired Parameter File

In [17]:
# Read the calibration parameter file into a DataFrame
cal_param = pd.read_csv('../PRMS_pothole_par.txt',encoding='utf-16', sep= '\t', header= 0, index_col=0)
#cal_param.rename(columns={cal_param.columns[0]: 'low_bound'}, inplace=True)
#cal_param.rename(columns={cal_param.columns[1]: 'upper_bound'}, inplace=True)

In [18]:
# Read Par File into a numpy array to convert to Paired    
with open(file_path, 'r') as file:
    lines = file.readlines()
par_paired = np.array([line.strip() for line in lines])

Define river segment list

In [19]:
# find upstream nseg index values which can be used to write nseg parameters
nseg= find_nseg(file_path, target_ID) 

In [20]:
len(nseg)

8

### Add previous segments

In [21]:
# find previous or upstream gauge
# previous_nseg1= find_nseg(file_path, upstream_ID_1)
# previous_nseg2= find_nseg(file_path, upstream_ID_2)

In [22]:
# remove previous gauge
# nseg= remove_duplicates(nseg, previous_nseg1)
# nseg= remove_duplicates(nseg, previous_nseg2)

In [23]:
# combine gauge nseg lists
#nseg= combine_nseg(nseg,nseg)
# nseg

### Find nhru and write paired parameter file

In [24]:
nhru= find_nhru(file_path, nseg)

In [25]:
len(nhru)

16

In [26]:
par_paired= write_nseg(par_paired, cal_param, nseg, updated_strings)

In [27]:
par_paired= write_nhru(par_paired, cal_param, nhru, updated_strings)

In [28]:
monthly, monthly_paired= write_alt_input(par_paired, cal_param, nhru, updated_strings)

### Obsin

In [29]:
# replace inflow to segment with observed flow
# par_paired=obsin_segment(par_paired, 21, 0)
# par_paired=obsin_segment(par_paired, 353, 5)

### Write ostin

In [31]:
ostin= write_ostin_table(cal_param, updated_strings) 

In [32]:
ostin

,Original_Value,Low_Bound,Upper_Bound,1,2,3
Updated_String,,,,,,
___carea_max521_A,0.161881,0.1,0.9,none,none,none
___carea_max522_A,0.188090,0.1,0.9,none,none,none
___carea_max494_A,0.184166,0.1,0.9,none,none,none
___carea_max497_A,0.175683,0.1,0.9,none,none,none
___carea_max495_A,0.229220,0.1,0.9,none,none,none
...,...,...,...,...,...,...
___adjust_tmin_cbh_adj_527_A,1.000000,−3,3.0,none,none,none
___adjust_tmin_cbh_adj_549_A,1.000000,−3,3.0,none,none,none
___adjust_tmin_cbh_adj_568_A,1.000000,−3,3.0,none,none,none


In [33]:
# Custom header and footer strings
top_half = """#Basic Configuration for Ostrich Program

#Essential variables
ProgramType ParallelDDS
ModelExecutable ./run-adj.sh
ModelSubdir mod_
ObjectiveFunction GCOP
OstrichWarmStart no

#Template File Configuration
BeginFilePairs
data/paired_myparam.param ; data/myparam.param
data/paired_monthly.txt ; data/monthly.txt
EndFilePairs

BeginExtraFiles
EndExtraFiles

BeginExtraDirs
data
EndExtraDirs


BeginParams"""

In [34]:
bottom_half= """EndParams

#Output variables to analyze
BeginResponseVars
#name   filename            keyword         line    col     token
KGE ./data/average_kge.txt ; OST_NULL 0 1 ' '
EndResponseVars

BeginGCOP
CostFunction KGE
PenaltyFunction APM
EndGCOP

#DDS algorithm setup
BeginParallelDDSAlg
PerturbationValue 0.2
MaxIterations 50000
UseInitialParamValues
EndParallelDDSAlg
"""


In [30]:
len(updated_strings)

592

### Calibration Outputs

In [35]:
# Write DataFrames to text files
monthly.to_csv('./data/monthly.txt', sep=' ', index=True)
monthly_paired.to_csv('./data/paired_monthly.txt', sep=' ', index=True)

In [36]:
# Save the parpaired to the specified file path
np.savetxt(output_file_path, par_paired, fmt='%s')

In [37]:
# Write the DataFrame to a text file with header and footer strings
with open(ostin_filepath, 'w') as file:
    file.write(top_half + '\n')
    ostin.to_csv(file, header=False, sep=' ')  # You can change the separator if needed
    file.write(bottom_half)

### Update Results

In [38]:
    # Read filepaths as filepath array as a NumPy array
    with open(cal_result, 'r') as file:
        lines = file.readlines()
    cal_result = np.array([line.strip() for line in lines])
    
        # Read filepaths as filepath array as a NumPy array
    with open(final_par, 'r') as file:
        lines = file.readlines()
    final_par = np.array([line.strip() for line in lines])

FileNotFoundError: [Errno 2] No such file or directory: './mod_0/data/myparam.param'

In [ ]:
# copy nseg cal results
par= update_nseg_results(cal_result, filter_variables_coef_variation, nseg, final_par)

In [ ]:
par= update_nhru_results(cal_result, filter_variables_coef_variation, nhru, final_par)

In [ ]:
par= update_nmonths_results(cal_result, filter_variables_coef_variation, nhru, final_par)

np.savetxt(final_par_output, par, fmt='%s')